In [1]:
from scienceworld import ScienceWorldEnv

from sources.agent.agent import BDIAgent
from sources.agent.scienceworld import parse_observation
from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [2]:
hg_model_hub_name = "alisawuffles/roberta-large-wanli"
nli_model = NLIModel(hg_model_hub_name)

In [3]:
# MAIN GOAL
main_plan = """
    IF your goal is to boil water THEN
        move to kitchen,
        pick up thermometer,
        get metal pot,
        fill metal pot with water,
        heat water on stove,
        focus on substance in metal pot
"""

# get metal pot
subplan_a = """
    IF your goal is to get metal pot CONSIDERING you are in the kitchen AND you see a cupboard THEN
        open cupboard,
        pick up metal pot
"""

# fill metal pot with water
subplan_b = """
    IF your goal is to fill metal pot with water CONSIDERING you have metal pot in your inventory AND you see a sink THEN
        move metal pot to sink,
        activate sink,
        deactivate sink,
        pick up metal pot
"""

# heat water on stove
subplan_c = """
    IF your goal is to heat water on stove CONSIDERING you have metal pot with water in your inventory AND you see a stove THEN
        move metal pot to stove,
        activate stove,
        wait,
        use thermometer on metal pot,
        use thermometer on metal pot,
        focus on substance in metal pot,
        wait,
        use thermometer on metal pot
"""

all_plans = [main_plan, subplan_a, subplan_b, subplan_c]

pl = PlanLibrary()
pl.load_plans_from_strings(all_plans) # load plans from strings above
pl.load_plans_from_file("plans.txt") # load plans from file
print(pl.plans.keys())

dict_keys(['boil water', 'get metal pot', 'fill metal pot with water', 'heat water on stove', 'move to art studio', 'move to bedroom', 'move to greenhouse', 'move to kitchen', 'move to living room', 'move to workshop', 'move to outside', 'move to foundry', 'move to bathroom', 'move to hallway'])


In [4]:
env = ScienceWorldEnv("", "", envStepLimit=100)

randVariationIdx = 0
task_type = 'boil'
root_event = 'boil water'
env.load('boil', randVariationIdx)

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=root_event, valid_actions=info['valid'])

Task Name: boil variation 0
Task Description: Your task is to boil water. For compounds without a boiling point, combusting the substance is also acceptable. First, focus on the substance. Then, take actions that will cause it to change its state of matter.


In [5]:
def step_function(action: str) -> State:
    observation, reward, isCompleted, info = env.step(action)
    print(observation)
    updated_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=root_event, valid_actions=info['valid'])
    return updated_state

agent = BDIAgent(plan_library=pl, nli_model=nli_model)
agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


The door is now open.
You move to the kitchen.
You move the thermometer to the inventory.
The cupboard is now open.
You move the metal pot to the inventory.
You move the metal pot to the sink.
The sink is now activated.
The sink is now deactivated.
You move the metal pot to the inventory.
You move the metal pot to the stove.
The stove is now activated.
You decide to wait for 10 iterations.
the thermometer measures a temperature of 156 degrees celsius
the thermometer measures a temperature of 161 degrees celsius
You focus on the water.
You decide to wait for 10 iterations.
the thermometer measures a temperature of 200 degrees celsius
Event focus on substance in metal pot is not a subgoal nor an action.
Completed keys: 
----------------------------------------------------------------------------------------------------
Sequential Subgoals:
----------------------------------------------------------------------------------------------------
0	true	                                GoalFind	f